In [ ]:
!pip install transformers datasets evaluate rouge_score


In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/filtered_summaries.csv")[["text", "summary"]].dropna()
dataset = Dataset.from_pandas(df)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], max_length=1024, truncation=True, padding="max_length")
    targets = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-mental-health",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=8,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)


In [ ]:
import evaluate
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer

rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=decoded_preds, references=decoded_labels)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-9-998690490.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import logging
logging.getLogger("wandb").setLevel(logging.ERROR)


trainer.train()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,11.378300,8.316099,0.306115,0.160329,0.270988,0.270979
2,7.829700,4.504869,0.326305,0.178304,0.284973,0.284898
3,4.704800,3.333776,0.332171,0.192475,0.297229,0.296296
4,3.606900,2.732225,0.367548,0.227073,0.331464,0.331539
5,3.189500,2.352257,0.363373,0.228880,0.328662,0.328891
6,2.730900,2.086742,0.376988,0.242525,0.345973,0.343950
7,2.518100,1.931199,0.378039,0.246139,0.346988,0.345222
8,2.422700,1.860925,0.368316,0.239960,0.339842,0.336950


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=152, training_loss=4.582854766594736, metrics={'train_runtime': 133.392, 'train_samples_per_second': 4.558, 'train_steps_per_second': 1.139, 'total_flos': 370719766609920.0, 'train_loss': 4.582854766594736, 'epoch': 8.0})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
model.push_to_hub("bart-mental-health-summarizer")
tokenizer.push_to_hub("bart-mental-health-summarizer")


Uploading...:   0%|          | 0.00/558M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/marku007/bart-mental-health-summarizer/commit/b8d4510f39ca8e39acd8b7780fa4aefda2d95a90', commit_message='Upload tokenizer', commit_description='', oid='b8d4510f39ca8e39acd8b7780fa4aefda2d95a90', pr_url=None, repo_url=RepoUrl('https://huggingface.co/marku007/bart-mental-health-summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='marku007/bart-mental-health-summarizer'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="marku007/bart-mental-health-summarizer")

sample_text = """Ive noticed that a lot of the time when im depressed ill generally get suicidal thoughts. Ive never attempted it, but I feel like the only reason I dont do it in these bouts of depression is because I know my parents would be sad and because I love my dog. I know it would break their hearts and ruin their whole lives if I did do it, and it would hurt some of the people at my work for sure, but it would just make everything so much easier. Obviously these arent normal thoughts to people who are always happy with everything going right in their lives, but im curious out of those who are depressed is it a normal or common thought? Even if you dont attempt to go through with it or plan to, just the thought of it."""

summary = summarizer(
    sample_text,
    max_length=128,
    min_length=16,
    do_sample=False,
    num_beams=4,
    no_repeat_ngram_size=3,
    early_stopping=True
)
print(summary[0]['summary_text'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Ive been having a lot of depression lately and I have been having suicidal thoughts.
